In [1]:
# import database

import database as db
database_name = 'db_weather'  #pw: bCCnw3b
db.set_up_connection(db.db, database_name, user='webscrapers')

postgres user password: ········


In [2]:
import pandas as pd
import numpy as np
import datetime

In [3]:
query = db.Station.select(lambda s: s.stations_id == '71')
pd.read_sql_query(query.get_sql(), db.conn_url)

,stations_id,von_datum,bis_datum,stationshoehe,geobreite,geolaenge,stationsname,bundesland
0,71,1986-11-01,2018-06-09,759,48.2156,8.9784,Albstadt-Badkap,Baden-Württemberg


In [4]:
query = db.DailyMeasurement.select(lambda s: s.stations_id=='71' and s.mess_datum < datetime.date(2018,6,1) and s.mess_datum > datetime.date(2018,5,1)) # web scraping data from 27/6 to 17/7
a = pd.read_sql_query(query.get_sql(), db.conn_url)['tmk']
b = np.asarray(a)
b

array([  8.8,  10.5,  13.5,  15.5,  16.8,  16.1,  16.8,  15.1,  10.1,
        11.8,  15.9,  12.4,  11. ,  11.7,   9.2,  10. ,  11.9,  11. ,
        12.2,  14.5,  13.4,  14.4,  15.4,  17. ,  18.6,  17.6,  19.6,
        17.5,  19.2,  17.7])

In [87]:
query = db.DailyPrediction.select(lambda s: True)
pd.read_sql_query(query.get_sql(), db.conn_url)

,id,website,city,date_of_acquisition,date_for_which_weather_is_predicted,temperature_max,temperature_min,wind_speed,humidity,precipitation_per,precipitation_l,wind_direction,condition,snow,uvi
0,111,The Weather Channel,BERLIN,2018-06-27,2018-06-27,24.0,16.0,18.0,53.0,20.0,None,NE,Partly Cloudy,0.0,None
1,112,The Weather Channel,BERLIN,2018-06-27,2018-06-28,27.0,15.0,23.0,52.0,0.0,None,NNE,Partly Cloudy,0.0,None
2,113,The Weather Channel,BERLIN,2018-06-27,2018-06-29,28.0,12.0,19.0,46.0,0.0,None,NNW,Clear,0.0,None
3,114,The Weather Channel,BERLIN,2018-06-27,2018-06-30,25.0,12.0,13.0,42.0,0.0,None,NE,Clear,0.0,None
4,115,The Weather Channel,BERLIN,2018-06-27,2018-07-01,27.0,12.0,14.0,38.0,0.0,None,ENE,Clear,0.0,None
5,116,The Weather Channel,BERLIN,2018-06-27,2018-07-02,27.0,14.0,11.0,38.0,0.0,None,E,Clear,0.0,None
6,117,The Weather Channel,BERLIN,2018-06-27,2018-07-03,28.0,17.0,11.0,41.0,0.0,None,SE,Clear,0.0,None
7,118,The Weather Channel,BERLIN,2018-06-27,2018-07-04,29.0,18.0,11.0,42.0,10.0,None,SSW,Clear,0.0,None
8,119,The Weather Channel,BERLIN,2018-06-27,2018-07-05,28.0,18.0,14.0,50.0,10.0,None,W,Partly Cloudy,0.0,None
9,120,The Weather Channel,BERLIN,2018-06-27,2018-07-06,26.0,16.0,14.0,51.0,20.0,None,WNW,Clear,0.0,None


In [55]:
dateT = pd.read_sql_query(query.get_sql(), db.conn_url).date_of_acquisition[0]
dateT

Timestamp('2018-06-27 00:00:00')

In [56]:
query = db.DailyPrediction.select(lambda s: s.date_of_acquisition == date)
pd.read_sql_query(query.get_sql(), db.conn_url)
#db.query_to_dataframe(query)

KeyError: 'p1'

In [13]:
def get_predictions(start,end,days_ahead=1,data_type='average_temperature'):
    """Extracts predictions from the database for specified days ahead in the specified time period
    Input:
    - start : starting date we want predictions for (date for which the weather was predicted). 
     Should be in datetime.date format
    - end : ending date we want predictions for
    - days_ahead : how many days ahead were the predictions made
    - data_type : data type we are interested in. Could be 'average_temperature',
    'temperature_max', 'temperature_min', 'wind_speed', 'humidity', 'precipitation_per', 'snow' 
    Ouput:
    - pred : array of predicted time series
    - real : array of real time series"""
    
    start_date = start - datetime.timedelta(days=days_ahead)
    prediction_date = start
    
    pred = []
    
    # could do it with one query maybe
    # query = db.DailyPrediction.select(lambda s: s.date_for_which_weather_is_predicted < end and s.date_for_which_weather_is_predicted == s.date_of_acquisition + datetime.timedelta(days=days_ahead))
        
    while (prediction_date != end):
        query = db.DailyPrediction.select(lambda s: s.date_of_acquisition == start_date and s.date_for_which_weather_is_predicted == prediction_date)
        if data_type == 'average_temperature':
            pred.append((query['temperature_max'] + query['temperature_min'])/2)
        else:
            pred.append(query.data_type)
        start_date += datetime.timedelta(days=1)
        prediction_date += datetime.timedelta(days=1)
    
    pred = np.asarray(pred)
    return pred

In [10]:
def get_measurements(start,end,station_id='71',data_type='tmk'):
    """Extracts measurements from the database in the specified time period
    Input:
    - start : starting date we want measurements for. Should be in datetime.date format
    - end : ending date we want measurements for
    - data_type : data type we are interested in. For possible arguments and their explanation look here:
    https://github.com/BCCN-Prog/dwd_2018/tree/master/dwd_descriptions
    Ouput:
    - real : array of real time series"""
    
    query = db.Station.select(lambda s: s.stations_id == station_id)
    data_start = pd.read_sql_query(query.get_sql(), db.conn_url)['von_datum'][0]
    data_end = pd.read_sql_query(query.get_sql(), db.conn_url)['bis_datum'][0]
    
    if (start < data_start or end > data.end):
        print('Data in specified period not available!')
        return 0
    
    query = db.DailyMeasurement.select(lambda s: s.stations_id==station_id and s.mess_datum < end and s.mess_datum > start)
    real = pd.read_sql_query(query.get_sql(), db.conn_url)[data_type]
    real = np.asarray(real)
    
    return real

In [11]:
real = get_measurements(datetime.date(2018,5,1),datetime.date(2018,7,1))

KeyError: 'p1'

In [7]:
query = db.Station.select(lambda s: s.stations_id == '71')
data_start = pd.read_sql_query(query.get_sql(), db.conn_url)['von_datum'][0]
data_end = pd.read_sql_query(query.get_sql(), db.conn_url)['bis_datum'][0]

In [8]:
data_start

datetime.date(1986, 11, 1)

In [9]:
data_end

datetime.date(2018, 6, 9)